# Generator

## Generator untuk Retrieval Process

In [1]:
from haystack import Pipeline, component
from haystack.components.builders import PromptBuilder
from haystack_integrations.components.retrievers.mongodb_atlas import MongoDBAtlasEmbeddingRetriever
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.routers import ConditionalRouter
from haystack_integrations.document_stores.mongodb_atlas import MongoDBAtlasDocumentStore
import os
from haystack.components.generators import OpenAIGenerator
from haystack.utils import Secret
from typing import List
from haystack.dataclasses import Document

d:\Enviroments\anaconda3\envs\day45-haystack\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Define Connection Environment

In [2]:
mongo_connection = Secret.from_token(os.getenv("MONGO_CONNECTION_STRING"))

In [3]:
openai_api_key = Secret.from_token(os.getenv("OPENAI_API_KEY"))

membuat pipeline

In [4]:
document_store_products = MongoDBAtlasDocumentStore(
    database_name="depato_store",
    collection_name="products",
    vector_search_index="vector_index",
    full_text_search_index="search_index",
    mongo_connection_string=mongo_connection
)
document_store_common = MongoDBAtlasDocumentStore(
    database_name="depato_store",
    collection_name="common_information",
    vector_search_index="vector_index_common",
    full_text_search_index=None,
    mongo_connection_string=mongo_connection
)

In [5]:
TEMPLATE_PRODUCTS = """"
You are a shop assiistant that helps users find the best products in a shopping mall.
You will be give a query and list of products. Your task is to generate a list of products that best match the query.
The query is: {{query}}
USE THE SAME LANGUAGE AS THE QUERY!
The output should be a list of products in the following format:


.  
Price: 
Material: 
Category: 
Brand: 
Recommendation: 

From the format above, you should pay attention to the following:
1.  should be a short summary of the query.
2.  should be a number starting from 1.
3.  should be the name of the product, this product name can be found from the product_name field.
4.  should be the price of the product, this product price can be found from the product_price field.
5.  should be the material of the product, this product material can be found from the product_material field.
6.  should be the category of the product, this product category can be found from the product_category field.
7.  should be the brand of the product, this product brand can be found from the product_brand field.
8.  should be the recommendation of the product, you should give a recommendation why this product is recommended, please pay attentation to the product_content field. 


You should only return the list of products that best match the query, do not return any other information.
the products are:
{% for product in documents %}
===========================================================
{{loop.index + 1}}. product_name: {{ product.meta.title }}
product_price: {{ product.meta.price }}
product_material: {{ product.meta.material }}
product_category: {{ product.meta.category }}
product_brand: {{ product.meta.brand }}
product_content: {{ product.content}}
{% endfor %}

===========================================================

Answer:

"""

In [6]:
TEMPLATE_BUILDER_ROUTER = """
You are a routing classifier.

Return EXACTLY ONE of the following strings:
products
common

STRICT RULES:
- DO NOT add quotes
- DO NOT add punctuation
- DO NOT add explanation
- DO NOT add extra words
- ONLY return exactly: products OR common

Examples:
Query: What is the price of this bag
Answer: products

Query: How do I return my order
Answer: common

Query: Does this bag come in red
Answer: products

Query: I want a refund for my last order
Answer: common

User Query:
{{ query }}

Routing rules:
- Return products if the query is about product details, specs, stock, brand, variants, price, warranty, colors, etc.
- Return common for shipping, returns, refunds, payment, policies, help, store info.
"""



In [7]:
TEMPLATE_COMMON = """
Answer the user's question using only the provided context.
Maintain the same language as the question.

Context:
{{ context | map(attribute='content') | join(" ") | replace("\n", " ") }}

Question: {{ query }}

Instructions:
1. Only use information from the context to answer.
2. If the context does not contain the required Context, respond with:
   "I'm sorry, I can't answer that right now."
3. Keep the answer concise and clear.
"""


In [8]:
routes = [
    {
        "condition": "{{ replies[0] == 'products'}}", 
        "output": ["{{ query_embedding }}","{{ query }}"],             
        "output_name": ["products_route_embedding" , "products_route_query"],        
        "output_type": [list[float], str]             
    },
    {
        "condition": "{{ replies[0] == 'common'}}", 
        "output": ["{{ query_embedding }}","{{ query }}"],
        "output_name": ["common_route_embedding" , "common_route_query"],
        "output_type": [list[float], str]
    }
]
router = ConditionalRouter(routes)

In [9]:
pipeline = Pipeline()

# --- 1. Komponen (Routing) ---
pipeline.add_component("embedder", SentenceTransformersTextEmbedder())
pipeline.add_component("prompt_builder_router", PromptBuilder(template=TEMPLATE_BUILDER_ROUTER))
pipeline.add_component("decision", OpenAIGenerator(model="gpt-4.1", api_key=openai_api_key))
pipeline.add_component("router", router) 

# --- 2. Alur "Products" ---
pipeline.add_component("retriever_products", MongoDBAtlasEmbeddingRetriever(
    document_store=document_store_products, 
    top_k=10
))
pipeline.add_component("prompt_builder_products", PromptBuilder(template=TEMPLATE_PRODUCTS))
pipeline.add_component("generator_products", OpenAIGenerator(model="gpt-4.1", api_key=openai_api_key))

# --- 3. Alur "Common" ---
pipeline.add_component("retriever_common", MongoDBAtlasEmbeddingRetriever(
    document_store=document_store_common, 
    top_k=6
))
pipeline.add_component("prompt_builder_common", PromptBuilder(template=TEMPLATE_COMMON))
pipeline.add_component("generator_common", OpenAIGenerator(model="gpt-4.1", api_key=openai_api_key))

PromptBuilder has 1 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`.
PromptBuilder has 2 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`.
PromptBuilder has 2 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`.


In [10]:
# --- 1. Koneksi Input & Routing ---
pipeline.connect("embedder.embedding", "router.query_embedding")
pipeline.connect("prompt_builder_router.prompt", "decision.prompt")
pipeline.connect("decision.replies", "router.replies")

# --- 2. Koneksi Alur "Products" ---
pipeline.connect("router.products_route_embedding", "retriever_products.query_embedding")
pipeline.connect("router.products_route_query", "prompt_builder_products.query")
pipeline.connect("retriever_products.documents", "prompt_builder_products.documents")
pipeline.connect("prompt_builder_products.prompt", "generator_products.prompt")

# --- 3. Koneksi Alur "Common" ---
pipeline.connect("router.common_route_embedding", "retriever_common.query_embedding")
pipeline.connect("router.common_route_query", "prompt_builder_common.query")
pipeline.connect("retriever_common.documents", "prompt_builder_common.context")
pipeline.connect("prompt_builder_common.prompt", "generator_common.prompt")

🚅 Components
  - embedder: SentenceTransformersTextEmbedder
  - prompt_builder_router: PromptBuilder
  - decision: OpenAIGenerator
  - router: ConditionalRouter
  - retriever_products: MongoDBAtlasEmbeddingRetriever
  - prompt_builder_products: PromptBuilder
  - generator_products: OpenAIGenerator
  - retriever_common: MongoDBAtlasEmbeddingRetriever
  - prompt_builder_common: PromptBuilder
  - generator_common: OpenAIGenerator
🛤️ Connections
  - embedder.embedding -> router.query_embedding (list[float])
  - prompt_builder_router.prompt -> decision.prompt (str)
  - decision.replies -> router.replies (list[str])
  - router.products_route_embedding -> retriever_products.query_embedding (list[float])
  - router.products_route_query -> prompt_builder_products.query (str)
  - router.common_route_embedding -> retriever_common.query_embedding (list[float])
  - router.common_route_query -> prompt_builder_common.query (str)
  - retriever_products.documents -> prompt_builder_products.documents (L

In [11]:
query="Barang saya ga sesuai!"
hasil = pipeline.run(
    data={
        "embedder":{"text": query},
        "prompt_builder_router": {"query": query},   
        "router" : {"query" : query}     
    },
    include_outputs_from=["router"]
)
print(hasil["router"])

Batches: 100%|██████████| 1/1 [00:00<00:00,  2.18it/s]


{'common_route_embedding': [0.028826454654335976, 0.045247405767440796, -0.003094836138188839, -0.003995463717728853, 0.04015699401497841, -0.004510172177106142, -0.008309008553624153, 0.024582315236330032, 0.04763161763548851, 0.0362996943295002, -0.03666418418288231, 0.01594361662864685, 0.00630308547988534, 0.09306523948907852, 0.028949378058314323, -0.053689222782850266, 0.016398873180150986, -0.0303840059787035, 0.13258080184459686, -0.0252057071775198, 0.002009607618674636, 0.007616644259542227, 0.008470526896417141, 0.020900556817650795, 0.02097140997648239, 0.0009628190309740603, 0.012843937613070011, -0.05677950754761696, -0.004667496308684349, -0.04351968690752983, 0.007980753667652607, 0.008595763705670834, -0.023941075429320335, 0.015174761414527893, 2.1899006696912693e-06, -0.026067981496453285, 0.048577044159173965, 0.015359302051365376, -0.033286236226558685, 0.04151774197816849, 0.0036995254922658205, 0.0038883357774466276, -0.009274719282984734, 0.016071483492851257, 0

In [ ]:
query="Barang saya ga sesuai!"
hasil = pipeline.run(
    data={
        "embedder":{"text": query},
        "prompt_builder_router": {"query": query},   
        "router" : {"query" : query}     
    },
    include_outputs_from=["prompt_builder_common"]
)
print(hasil["generator_common"]["replies"][0])

Batches: 100%|██████████| 1/1 [00:00<00:00, 22.05it/s]


Jika barang yang Anda terima tidak sesuai, Anda dapat mengajukan pengembalian barang dalam waktu 7–14 hari setelah diterima, asalkan produk masih dalam kondisi asli, belum digunakan, dan kemasannya lengkap. Produk digital, pakaian dalam, serta produk yang sudah digunakan tidak dapat dikembalikan. Proses refund memakan waktu 3–5 hari kerja setelah barang diterima dan diverifikasi.


In [17]:
query="Rekomendasi baju wanita"
hasil = pipeline.run(
    data={
        "embedder":{"text": query},
        "prompt_builder_router": {"query": query},   
        "router" : {"query" : query}     
    },
    include_outputs_from=["decision"]
)
print(hasil["generator_products"]["replies"][0])

Batches: 100%|██████████| 1/1 [00:00<00:00, 14.42it/s]


Rekomendasi baju wanita terbaik di mall

1. Nama Produk: buXsbaum Girlie T-Shirt Bosnia and Herzegovina  
   Price: 18.99  
   Material: Cotton  
   Category: Tops  
   Brand: buXsbaum  
   Recommendation: Kaos wanita dengan desain menarik ini sangat cocok untuk kamu yang ingin tampil beda. Terbuat dari 100% katun yang nyaman dipakai seharian. Slim fit sehingga pas di badan dan tersedia dalam berbagai ukuran.

2. Nama Produk: buXsbaum Girlie T-Shirt Jiujitsu  
   Price: 18.99  
   Material: Cotton  
   Category: Tops  
   Brand: buXsbaum  
   Recommendation: T-shirt wanita ini dibuat dari katun berkualitas, nyaman dipakai, serta desainnya unik dan trendy. Cocok digunakan untuk aktivitas santai maupun casual.

3. Nama Produk: buXsbaum Girlie T-Shirt Mexiko  
   Price: 14.555  
   Material: Cotton  
   Category: Tops  
   Brand: buXsbaum  
   Recommendation: Kaos wanita ini harganya terjangkau dan tetap mengutamakan kenyamanan. Bahan katunnya halus dan potongannya slim fit, membuat penam